In [ ]:
#Agregamos las librerías necesarias
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import plotly as pl
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from geomet import wkt
from shapely import wkt as shpwkt, geometry
from shapely.geometry import mapping, Point, Polygon
import seaborn as sns

%matplotlib inline

---
# Inicio

In [ ]:
#Cargamos los datos y vemos los primeros registros
data = pd.read_csv('data/properatti.csv')
data.head(5)

In [ ]:
''' Vemos las columnas '''

data.info(verbose=True,null_counts=True)

In [ ]:
registros_pre_capital = data.shape[0]
print("Cantidad de registros del dataset:",registros_pre_capital)

---
Eliminamos las columnas que no aportan informacion, como operation es sell la quitamos, expensas también no aporta información y es un dato que nunca se completa correctamente, country son todos de argentina no aporta nada. Image_thumbnail no aporta información ni la dirección url

In [ ]:
data = data.drop(['Unnamed: 0','operation','country_name','expenses','image_thumbnail', 'properati_url'], axis=1)

---
# Redimensión del Dataset
### Achicamos el dataset a Capital Federal

#### Rellenamos los nulls para place_name

In [ ]:
print("Place names null antes: ", data.place_name.isnull().sum())

''' completar los places names '''
data.loc[data.place_name.isnull(),'place_name'] = data.loc[data.place_name.isnull()].place_with_parent_names.str.split('|', expand=True)[3]

#Relleno manualmente un dato que está suelto
mask=(data.place_name=='coordenadas 34.255511')
data.loc[mask,'place_name'] = 'Rincón de Milberg'

'''place names nulls despues de completar'''

print("Place names null después: ", data.place_name.isnull().sum())

#### Cargamos el dataset del gobierno de la ciudad con los puntos WKT

In [ ]:
gcba = pd.read_csv('./data/barrios.csv')

#Convertimos los puntos WKT a Polygon con shapely
gcba["WKT"] = gcba["WKT"].apply(shpwkt.loads)
gcba.head(3)

In [ ]:
#Achico el dataset quedándome solamente con aquellas filas que tengan place name o state name Capital Federal
data=data.loc[(data.state_name=='Capital Federal') | (data.place_name=='Capital Federal')]

#Elimino las filas que no tenga valores lat o lon
data=data.drop(data[(data['lat'].isnull()) | (data['lon'].isnull())].index)

#Reseteo el index
data.reset_index(inplace=True, drop=True)
#Agrego la columna Point para cada valor lat-lon
data['Point'] = data.apply(lambda x: Point(x['lon'], x['lat']), axis=1)

data.head(3)

In [ ]:
#Creo una columna donde tengo los barrios normalizados que me trae el nombre del barrio siguiendo el valor del dataset del gcba
def returnBarrio(point):
    try:
        return gcba['BARRIO'].loc[[i for i,wkt in enumerate(gcba.WKT) if wkt.contains(point)][0]]
    except:
        return np.nan

data['place_name_normalized'] = data.Point.apply(returnBarrio)
data.head(3)

In [ ]:
#Borro todos aquellos que tengan NaN en la columna place_name_normalized
#Elimino las filas que no tenga valores lat o lon
data=data.drop(data[(data['place_name_normalized'].isnull())].index)

#Reseteo el index
data.reset_index(inplace=True, drop=True)
data.head(3)

In [ ]:
print("Cantidad de registros del dataset:",registros_pre_capital)
cantidad_de_registros_capital_completo = data.shape[0]
print("Cantidad de registros solo de capital federal:",cantidad_de_registros_capital_completo)

In [ ]:
nulls = pd.DataFrame({'Columna':data.columns, 'Nulos':data.isnull().sum(),'No nulos':data.notnull().sum()})

# plot a Stacked Bar Chart using matplotlib
nulls.plot(
  x = 'Columna', 
  kind = 'barh', 
  stacked = True, 
  title = 'Nulos por columna', 
  mark_right = True)

---
# Relleno de nulls

### Currency

In [ ]:
#Revisamos en que moneda se encuentran expresadas las propiedades
data.currency.value_counts()

In [ ]:
#Chequeo anterior de nulos
print('Price aprox usd nulls antes: ',data.price_aprox_usd.isnull().sum())

#Extraemos de las descripciones los montos en dólares para completar dólares
patron = r'(\bU\$S|usd|uss|dól|dol)(\s)*(?P<miles>\d+)(\.)?(?P<unidades>\d*)'

#Guardamos los valores y lo asignamos en la columna price_aprox_usd
p_dls = data.loc[data.price_aprox_usd.isnull()].description.str.extract(patron, flags= re.IGNORECASE, expand = False)
p_dls['miles'] = pd.to_numeric(p_dls['miles'], errors= 'coerce')
p_dls['unidades'] = pd.to_numeric(p_dls['unidades'], errors= 'coerce')
data.loc[data.price_aprox_usd.isnull(),'price_aprox_usd'] = p_dls['miles']*1000 + p_dls['unidades']

#Chequeo posterior de nulos
print('Price aprox usd nulls después: ',data.price_aprox_usd.isnull().sum())

In [ ]:
#Extraemos de los títulos los montos en dólares para completar dólares
p_dls = data.loc[data.price_aprox_usd.isnull()].title.str.extract(patron, flags= re.IGNORECASE, expand = False)
p_dls['miles'] = pd.to_numeric(p_dls['miles'], errors= 'coerce')
p_dls['unidades'] = pd.to_numeric(p_dls['unidades'], errors= 'coerce')
data.loc[data.price_aprox_usd.isnull(),'price_aprox_usd'] = p_dls['miles']*1000 + p_dls['unidades']

#Chequeo posterior de nulos
print('Price aprox usd nulls después: ',data.price_aprox_usd.isnull().sum())

In [ ]:
#Exploramos las columnas de los precios para ver si hay datos no nulos

data.loc[data['price_aprox_usd'].isna()].loc[:,['price','price_aprox_local_currency','price_usd_per_m2','price_per_m2']].isna().sum()

In [ ]:
#Quitamos los precios nulos

data = data.drop(data[data['price_aprox_usd'].isna()].index)
data.isnull().sum()

### Rooms

In [ ]:
#Rooms nulos antes
print('Nulos antes: ',data.rooms.isnull().sum())

#Relleno con patron regex según la palabra ambiente
patron = '((?P<numero>\d|mono|dos|tres|cuatro|cinco|seis)\s*(?P<ambiente>amb))'
patron_regex = re.compile(patron,flags = re.IGNORECASE)

def convertRoomFromAmbiente(row):  
    try:
        resultado=None
        ambientes=0
        if (row['description'] and isinstance(row['description'],str)):
            resultado = patron_regex.search(row['description'])
        if ((resultado is None) and row['title'] and isinstance(row['title'],str)):
            resultado = patron_regex.search(row['title'])
        if (resultado is not None):
            if (resultado.group('numero2').lower()=='un'):
                ambientes+=1
            elif(resultado.group('numero2').lower()=='dos'):
                ambientes+=2
            elif(resultado.group('numero2').lower()=='tres'):
                ambientes+=3
            elif(resultado.group('numero2').lower()=='cuatro'):
                ambientes+=4
            elif(resultado.group('numero2').lower()=='cinco'):
                ambientes+=5
            elif(resultado.group('numero2').lower()=='seis'):
                ambientes+=6
            else:
                ambientes=int(resultado.group('numero'))
        return ambientes
    except:
        return np.nan
        
data.loc[data.rooms.isnull(),'rooms'] = data.loc[data.rooms.isnull()].apply(convertRoomFromAmbiente,axis=1)

#Relleno con patron regex según la palabra dormitorio o habitación
patron = '"((?P<numero>\d|un|dos|tres|cuatro|cinco|seis)(\s\w+)?\s(?P<dormitorios>dorm|hab|HAB|Hab))"'
patron_dormhab = re.compile(patron,flags = re.IGNORECASE)

def convertRoomFromDormHab(row):
    resultado = None
    if (row['description'] and isinstance(row['description'],str)):
        resultado = patron_dormhab.finditer(row['description'])
    if ((resultado is None) and row['title'] and isinstance(row['title'],str)):
        resultado = patron_dormhab.finditer(row['description'])
    if not(resultado is None):
        dormitorios=0
        for m in resultado:
            if m.group('numero') is None:
                dormitorios=np.nan
            else:
                if(m.group('numero').lower()=='un'):
                    dormitorios+=1
                elif(m.group('numero').lower()=='dos'):
                    dormitorios+=2
                elif(m.group('numero').lower()=='tres'):
                    dormitorios+=3
                elif(m.group('numero').lower()=='cuatro'):
                    dormitorios+=4
                elif(m.group('numero').lower()=='cinco'):
                    dormitorios+=5
                elif(m.group('numero').lower()=='seis'):
                    dormitorios+=6
                else:
                    dormitorios+=int(m.group('numero'))
    if not(np.isnan(dormitorios)) and dormitorios>0:
        return dormitorios
    else:
        return np.nan

data.loc[data.rooms.isnull(),'rooms'] = data.loc[data.rooms.isnull()].apply(convertRoomFromDormHab,axis=1)

#Rooms nulos despues de limpiar
print('Nulos después: ',data.rooms.isnull().sum())

### Piso
Para los departamentos

In [ ]:
#Cuantos departamentos tienen el piso en null
len(data.loc[(data.property_type == 'apartment') & (data.floor.isnull())].index)

In [ ]:
patron_floor = re.compile("(?P<piso0>planta baja|PB)|(?P<piso1>1er piso|primer piso|piso 1|1º piso|piso 1º)|(?P<piso2>2do piso|segundo piso|piso 2|2º piso|piso 2º)|(?P<piso3>3er piso|tercer piso|piso 3|3º piso|piso 3º)|(?P<piso4>4to piso|cuarto piso|piso 4|4º piso|piso 4º)|(?P<piso5>5to piso|quinto piso|piso 5|5º piso|piso 5º)|(?P<piso6>6to piso|sexto piso|piso 6|6º piso|piso 6º)|(?P<piso7>7to piso|séptimo piso|piso 7|7º piso|piso 7º)|(?P<piso8>8vo piso|octavo piso|piso 8|8º piso|piso 8º)|(?P<piso9>9no piso|noveno piso|piso 9|9º piso|piso 9º)|(?P<piso10>10mo piso|décimo piso|piso 10|10º piso|piso 10º)|(?P<piso11>11vo piso|onceavo piso|piso 11|11º piso|piso 11º)|(?P<piso12>12vo piso|doceavo piso|piso 12|12º piso|piso 12º)|(?P<piso13>13vo piso|treceavo piso|piso 13|13º piso|piso 13º)|(?P<piso14>14vo piso|cartoceavo piso|piso 14|14º piso|piso 14º)|(?P<piso15>15vo piso|quinceavo piso|piso 15|15º piso|piso 15º)|(?P<piso16>16to piso|decimosexto piso|piso 16|16º piso|piso 16º)|(?P<piso17>17to piso|decimoséptimo piso|piso 17|17º piso|piso 17º)|(?P<piso18>18vo piso|decimooctavo piso|piso 18|18º piso|piso 18º)|(?P<piso19>19vo piso|decimonoveno piso|piso 19|19º piso|piso 19º)|(?P<piso20>20to piso|vigésimo piso|piso 20|20º piso|piso 20º)",flags = re.IGNORECASE)

def extractFloor(row):
    match = None
    if (row['description'] and isinstance(row['description'],str) ):
        match = patron_floor.search(row['description'])
    if (match is None) and row['title'] and isinstance(row['title'],str):
        match = patron_floor.search(row['title'])
    if match is None:
        return np.nan
    elif match.group('piso0') is not None:
        return 0.0
    elif match.group('piso1') is not None:
        return 1.0
    elif match.group('piso2') is not None:
        return 2.0
    elif match.group('piso3') is not None:
        return 3.0
    elif match.group('piso4') is not None:
        return 4.0
    elif match.group('piso5') is not None:
        return 5.0
    elif match.group('piso6') is not None:
        return 6.0
    elif match.group('piso7') is not None:
        return 7.0
    elif match.group('piso8') is not None:
        return 8.0
    elif match.group('piso9') is not None:
        return 9.0
    elif match.group('piso10') is not None:
        return 10.0
    elif match.group('piso11') is not None:
        return 11.0
    elif match.group('piso12') is not None:
        return 12.0
    elif match.group('piso13') is not None:
        return 13.0
    elif match.group('piso14') is not None:
        return 14.0
    elif match.group('piso15') is not None:
        return 15.0
    elif match.group('piso16') is not None:
        return 16.0
    elif match.group('piso17') is not None:
        return 17.0
    elif match.group('piso18') is not None:
        return 18.0
    elif match.group('piso19') is not None:
        return 19.0
    elif match.group('piso20') is not None:
        return 20.0
    else:
        return np.nan

data.loc[(data.property_type == 'apartment') & (data.floor.isnull()),'floor'] = data.loc[(data.property_type == 'apartment') & (data.floor.isnull())].apply(lambda x: extractFloor(x),axis =1)

#Cuantos departamentos tienen el piso en null
len(data.loc[(data.property_type == 'apartment') & (data.floor.isnull())].index)

In [ ]:
data.rooms.value_counts()

### Superficie

In [ ]:
#Completamos ambas columnas de superficie
print("Antes")
print("surface_covered_in_m2 nulos:",data['surface_covered_in_m2'].isnull().sum())
print("surface_total_in_m2 nulos:",data['surface_total_in_m2'].isnull().sum()) 

data.loc[data.surface_covered_in_m2 > data.surface_total_in_m2,'surface_total_in_m2'] = data.loc[data.surface_covered_in_m2 > data.surface_total_in_m2,'surface_covered_in_m2']

data['surface_covered_in_m2'] = data['surface_covered_in_m2'].fillna(data['surface_total_in_m2'])
data['surface_total_in_m2'] = data['surface_total_in_m2'].fillna(data['surface_covered_in_m2'])

#Aquellas superficies totales menores a 10 m2 lo consideramos nulo
data.loc[data.surface_total_in_m2 < 5,'surface_total_in_m2'] = np.NaN

In [ ]:
print('Nulls antes para surface_total_in_m2: ',data.surface_total_in_m2.isnull().sum())

patron_meters = re.compile("(?P<Metros>\d+)(?P<tipo>metro|\smetro|mts|\smts|m2|\sm2)",flags = re.IGNORECASE)

def extractMeters(row):
    meters_match = None
    if ( row['description'] and isinstance(row['description'],str) ):
        meters_match = patron_meters.search(row['description'])
    if ((meters_match is None) and row['title'] and isinstance(row['title'],str)):
        meters_match = patron_meters.search(row['title'])
    if (meters_match is not None):
        return meters_match.groups()[0]
    return None

data.loc[data.surface_total_in_m2.isnull(),'surface_total_in_m2'] = data.loc[data.surface_total_in_m2.isnull()].apply(extractMeters,axis=1)

data.loc[data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.notnull(),'surface_total_in_m2'] = data.loc[data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.notnull()].apply(lambda x: x['surface_covered_in_m2'],axis=1)

data.loc[data.surface_total_in_m2.isnull(),'surface_total_in_m2'] = data.loc[data.surface_total_in_m2.isnull()].surface_total_in_m2.fillna(value=np.nan)
data.loc[:,'surface_total_in_m2'] = data.loc[:,'surface_total_in_m2'].astype("float")

''' cuantos quedaron'''
print('Nulls después para surface_total_in_m2: ',data.surface_total_in_m2.isnull().sum())

In [ ]:
data.loc[data.surface_total_in_m2 < 5,'surface_total_in_m2'] = np.NaN
data.surface_total_in_m2.isnull().sum()

In [ ]:
#Completamos la columna price usd per m2 para los valores que rellenamos
data.loc[data.surface_total_in_m2.notnull() & data.price_aprox_usd.notnull() & data.price_usd_per_m2.isnull() & data.surface_total_in_m2 > 0,'price_usd_per_m2'] = data.loc[data.surface_total_in_m2.notnull() & data.price_aprox_usd.notnull() & data.price_usd_per_m2.isnull() & data.surface_total_in_m2 > 0].apply(lambda x: x['price_aprox_usd'] / x['surface_total_in_m2'],axis=1)

data.isnull().sum()

---
# Nuevas columnas
### Columna disposición
Creamos la columna disposición del departamento obtenida a traves de las descripciones o los títulos

In [ ]:
#Columna disposicion del departamento

patron_disposicion = re.compile("(?P<tipo>frente|lateral|contrafrente)",flags = re.IGNORECASE)

def disposicion(row):
    disposicion_match = None
    if ( row['description'] and isinstance(row['description'],str) ): 
        disposicion_match = patron_disposicion.search(row['description'])
    if ((patron_disposicion is None) and row['title'] and isinstance(row['title'],str)):
        disposicion_match = patron_disposicion.search(row['title'])
    if (disposicion_match is not None):
        return disposicion_match.groups()[0].lower()
    return None

data['disposicion'] = data.apply(lambda x:disposicion(x),axis = 1)
print(data['disposicion'].value_counts())
print("Total dataset:",data.shape[0])

### Columna A Estrenar
Creamos la columna A Estrenar obtenida a través de las descripciones o los títulos

In [ ]:
#Columna a estrenar
patron_a_estrenar = re.compile("(?P<tipo>estrenar)",flags = re.IGNORECASE)

def aestrenar(row):
    try:
        if (row['description'] and isinstance(row['description'],str) ):
            if (patron_a_estrenar.search(row['description']).group('tipo') is not None):
                return True
        if (row['title'] and isinstance(row['title'],str) ):
            if (patron_a_estrenar.search(row['title']).group('tipo') is not None):
                return True
    except:
        return False

data['aAstrenar'] = data.apply(lambda x:aestrenar(x),axis = 1);
data['aAstrenar'].value_counts()

In [ ]:
print("Cantidad de registros del dataset:",registros_pre_capital)
print("Cantidad de registros solo de capital federal:",cantidad_de_registros_capital_completo)
print("Cantidad de registros solo de capital federal luego de rellenar y filtrar:",data.shape[0])

---
# Análisis

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
data.describe()

## Outliers mediante Boxplot

In [ ]:
#Analizamos outliers mediante boxplot
sns.set_theme(style="whitegrid")
plt.figure(figsize=(15,12))
sns.boxplot(data=data, x="price_aprox_usd", y="property_type", orient="h")
plt.show()

In [ ]:
plt.figure(figsize=(15,12))
sns.boxplot(data=data, x="price_usd_per_m2", y="property_type", orient="h")
plt.show()

In [ ]:
plt.figure(figsize=(15,12))
sns.boxplot(data=data, x="surface_total_in_m2", y="property_type", orient="h")
plt.show()

In [ ]:
def is_outlier(s, quantiles=[.25, .75], thresholds=[-1.5, 1.5]):
    # change the thresholds to [-1.5, 1.5] to reflect IQR as per your question
    a, b = s.quantile(quantiles)
    iqr = b - a
    lo, hi = np.array(thresholds) * iqr + [a, b]
    if(lo<s.min()):
        lo=s.min()
    if(hi>s.max()):
        hi=s.max()
    return (s < lo) | (s > hi)

mask = data.groupby('property_type')['surface_total_in_m2'].apply(is_outlier)
test1 = data.loc[~mask]

plt.figure(figsize=(15,12))
sns.boxplot(data=test1, x="surface_total_in_m2", y="property_type", orient="h")
plt.show()

In [ ]:
test2=data[data.groupby("property_type").price_usd_per_m2.\
      transform(lambda x : (x<x.quantile(0.957))&(x>(x.quantile(0.05)))).eq(1)]
plt.figure(figsize=(15,12))
sns.boxplot(data=test2, x="price_usd_per_m2", y="property_type", orient="h")
plt.show()

In [ ]:
def is_outlier(s, quantiles=[.25, .75], thresholds=[-1.5, 1.5]):
    # change the thresholds to [-1.5, 1.5] to reflect IQR as per your question
    a, b = s.quantile(quantiles)
    iqr = b - a
    lo, hi = np.array(thresholds) * iqr + [a, b]
    if(lo<s.min()):
        lo=s.min()
    if(hi>s.max()):
        hi=s.max()
    return (s < lo) | (s > hi)

mask = data.groupby('property_type')['price_usd_per_m2'].apply(is_outlier)
test3 = data.loc[~mask]

plt.figure(figsize=(15,12))
sns.boxplot(data=test3, x="price_usd_per_m2", y="property_type", orient="h")
plt.show()

In [ ]:
data.loc[data.price_usd_per_m2 > 10000,'price_usd_per_m2'] = np.NaN

#Quitamos los que tienen 3 desvios de la media
data.loc[data['price_usd_per_m2']>=(data['price_usd_per_m2'].mean() + 3*data['price_usd_per_m2'].std()),'price_usd_per_m2'] = np.NaN
data.loc[data['price_aprox_usd']>=(data['price_aprox_usd'].mean() + 3*data['price_aprox_usd'].std()),'price_aprox_usd'] = np.NaN
data.loc[data['surface_total_in_m2']>=(data['surface_total_in_m2'].mean() + 3*data['surface_total_in_m2'].std()),'surface_total_in_m2'] = np.NaN
# intentamos completar con los otros campos'''
data.loc[data.surface_total_in_m2.notnull() & data.price_aprox_usd.notnull() & data.price_usd_per_m2.isnull() & data.surface_total_in_m2 > 0,'price_usd_per_m2'] = data.loc[data.surface_total_in_m2.notnull() & data.price_aprox_usd.notnull() & data.price_usd_per_m2.isnull() & data.surface_total_in_m2 > 0].apply(lambda x: x['price_aprox_usd'] / x['surface_total_in_m2'],axis=1)
data.loc[data.surface_total_in_m2.notnull() & data.price_aprox_usd.isnull() & data.price_usd_per_m2.notnull() & data.surface_total_in_m2 > 0,'price_aprox_usd'] = data.loc[data.surface_total_in_m2.notnull() & data.price_aprox_usd.isnull() & data.price_usd_per_m2.notnull() & data.surface_total_in_m2 > 0].apply(lambda x: x['price_usd_per_m2'] * x['surface_total_in_m2'],axis=1)
data.loc[data.surface_total_in_m2.isnull() & data.price_aprox_usd.notnull() & data.price_usd_per_m2.notnull() & data.price_usd_per_m2 > 0,'surface_total_in_m2'] = data.loc[data.surface_total_in_m2.isnull() & data.price_aprox_usd.notnull() & data.price_usd_per_m2.notnull() & data.price_usd_per_m2 > 0].apply(lambda x: x['price_aprox_usd'] / x['price_usd_per_m2'],axis=1)

#quitamos los registros sin info relevante
columnsUtils = ['surface_total_in_m2','price_aprox_usd', 'price_usd_per_m2']

data = data.dropna(axis =0 , how = 'any', subset = columnsUtils)
data = data.drop(data.loc[data.price_usd_per_m2 > 10000].index)

plt.figure(figsize=(15,12))
sns.boxplot(data=data, x="price_usd_per_m2", y="property_type", orient="h")
plt.show()

In [ ]:
data = data.drop(data.loc[(data.surface_total_in_m2>2000)].index)
plt.figure(figsize=(15,12))
sns.boxplot(data=data, x="surface_total_in_m2", y="property_type", orient="h")
plt.show()

In [ ]:
nulls2 = pd.DataFrame({'Columna':data.columns, 'Nulos':data.isnull().sum(),'No nulos':data.notnull().sum()})

# plot a Stacked Bar Chart using matplotlib
nulls2.plot(
  x = 'Columna', 
  kind = 'barh', 
  stacked = True, 
  title = 'Nulos por columna', 
  mark_right = True)

In [ ]:
df_tipo_prop = data.groupby('property_type').mean()['price_aprox_usd'].sort_values(ascending=False)[0:5]
graf = df_tipo_prop.sort_values().plot.barh(figsize=(14,5),fontsize=15, color='r')
graf.set_title("Precio promedio segun el tipo de propiedad", fontsize=10)
graf.set_ylabel("Tipo de propiedad", fontsize=18)
graf.set_xlabel("U$S", fontsize=18)
plt.show()

In [ ]:
dfzone = data.groupby('place_name').mean()['price_usd_per_m2'].sort_values(ascending=False)[0:10]
g = dfzone.sort_values().plot.barh(figsize=(14,5),color='b',fontsize=15, alpha=0.6);
g.set_title("Precio(usd/m2) promedio según el barrio", fontsize=20)
g.set_ylabel("Barrio", fontsize=18)
g.set_xlabel("U$S/m2", fontsize=18)
plt.show()

In [ ]:
dfzone = data.groupby('place_name').mean()['price_usd_per_m2'].sort_values(ascending=True)[0:10]
g = dfzone.sort_values().plot.barh(figsize=(14,5),color='b',fontsize=15, alpha=0.6);
g.set_title("Precio(usd/m2) promedio según el barrio", fontsize=20)
g.set_ylabel("Barrio", fontsize=18)
g.set_xlabel("U$S/m2", fontsize=18)
plt.show()

---
## Mapa de colores según precios

In [ ]:
#Creo un dataset reducido para agruparlo y calcular el valor medio por cada tipo de propiedad. Luego lo convierto en dataframe con unstack
datos_reducidos_precios = data.loc[:,["property_type","place_name_normalized","price_usd_per_m2"]].groupby(['place_name_normalized','property_type']).mean().unstack(1)

#Convierto el index en una columna
datos_reducidos_precios = datos_reducidos_precios.reset_index()
#Bajo el multilevel
datos_reducidos_precios.columns = ["".join(a) for a in datos_reducidos_precios.columns.to_flat_index()]
#Renombro las columnas de precios
datos_reducidos_precios.columns = ['Barrio', 'Precio promedio PH','Precio promedio departamento','Precio promedio casa','Precio promedio local']
datos_reducidos_precios.head()

In [ ]:
#Hacemos un merge con nuestros datos
capital_merge_precios = datos_reducidos_precios.merge(gcba, left_on="Barrio", right_on="BARRIO",how="left")
capital_merge_precios.head()

In [ ]:
#Armamos los polígonos para plotly
lista_polygonos=[ {'geometry': mapping(x.WKT),'id' : x.BARRIO } for i,x in capital_merge_precios.iterrows() ] 

In [ ]:
geojson_capital= {'type': 'FeatureCollection' ,
                  'features' : lista_polygonos }

fig = make_subplots(
    rows=2, cols=2,subplot_titles=['Precio promedio U$S PH','Precio promedio U$S Dpto','Precio promedio U$S Casas','Precio promedio U$S Locales'],
    specs=[[{"type": "choroplethmapbox"}, {"type": "choroplethmapbox"}], [{"type": "choroplethmapbox"}, {"type": "choroplethmapbox"}]],
)

fig.add_trace(px.choropleth_mapbox(capital_merge_precios,
                                  geojson=geojson_capital,
                                  locations='Barrio', 
                                  color='Precio promedio PH',
                                  opacity=0.5, height=500)['data'][0],
                                  row=1, col=1)

fig.add_trace(px.choropleth_mapbox(capital_merge_precios,
                                  geojson=geojson_capital, 
                                  locations='Barrio', 
                                  color='Precio promedio departamento',
                                  opacity=0.5, height=500,)['data'][0],
                                  row=1, col=2)

fig.add_trace(px.choropleth_mapbox(capital_merge_precios,
                                  geojson=geojson_capital, 
                                  locations='Barrio', 
                                  color='Precio promedio casa',
                                  opacity=0.5, height=500,)['data'][0],
                                  row=2, col=1)

fig.add_trace(px.choropleth_mapbox(capital_merge_precios,
                                  geojson=geojson_capital, 
                                  locations='Barrio', 
                                  color='Precio promedio local',
                                  opacity=0.5, height=500,)['data'][0],
                                  row=2, col=2)

fig.update_mapboxes(
        center = {"lat": -34.615, "lon": -58.45},
        zoom = 10,
        style='carto-positron',
 )

fig.update_layout(
    margin={"r":0,"t":20,"l":0,"b":0},
    height=950
)

fig.show()

In [ ]:
fig = px.choropleth_mapbox(capital_merge_precios, geojson=geojson_capital,
                      locations='Barrio', 
                      color='Precio promedio PH',
                      color_continuous_scale="OrRd",
                      range_color=(0, capital_merge_precios['Precio promedio PH'].max()),
                      mapbox_style="carto-positron",
                      opacity=0.5,
                      zoom=10.5,
                    labels = {'Precio promedio PH':'Precio promedio PH en U$D'},
                      center = {"lat": -34.61, "lon": -58.45})

fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(capital_merge_precios, geojson=geojson_capital,
                      locations='Barrio', 
                      color='Precio promedio departamento',
                      color_continuous_scale="OrRd",
                      range_color=(0, capital_merge_precios['Precio promedio departamento'].max()),
                      mapbox_style="carto-positron",
                      opacity=0.5,
                      zoom=10,
                    labels = {'Precio promedio departamento':'Precio promedio departamento en U$D'},
                      center = {"lat": -34.61, "lon": -58.45})
fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

---
## Mapa de colores según cantidad de propiedad

In [ ]:
#Creo un dataset reducido para agruparlo y calcular la cantidad por cada tipo de propiedad. Luego lo convierto en dataframe con unstack
datos_reducidos_propiedades = data.loc[:,["place_name_normalized","property_type"]].groupby(["place_name_normalized","property_type"]).size().unstack(1)

#Convierto el index en una columna
datos_reducidos_propiedades = datos_reducidos_propiedades.reset_index()
#Bajo el multilevel
datos_reducidos_propiedades.columns = ["".join(a) for a in datos_reducidos_propiedades.columns.to_flat_index()]

#Hacemos un merge con nuestros datos
capital_merge_propiedades = datos_reducidos_propiedades.merge(gcba, left_on="place_name_normalized", right_on="BARRIO",how="left")
capital_merge_propiedades.head()

In [ ]:
geojson_capital= {'type': 'FeatureCollection' ,
                  'features' : lista_polygonos }

fig = make_subplots(
    rows=2, cols=2,subplot_titles=['Cantidad de PH','Cantidad de Departamentos','Cantidad de Casas','Cantidad de Locales'],
    specs=[[{"type": "choroplethmapbox"}, {"type": "choroplethmapbox"}], [{"type": "choroplethmapbox"}, {"type": "choroplethmapbox"}]],
)

fig.add_trace(px.choropleth_mapbox(capital_merge_propiedades,
                                  geojson=geojson_capital,
                                  locations='BARRIO', 
                                  color='PH',
                                  opacity=0.5, height=500)['data'][0],
                                  row=1, col=1)

fig.add_trace(px.choropleth_mapbox(capital_merge_propiedades,
                                  geojson=geojson_capital, 
                                  locations='BARRIO', 
                                  color='apartment',
                                  opacity=0.5, height=500,)['data'][0],
                                  row=1, col=2)

fig.add_trace(px.choropleth_mapbox(capital_merge_propiedades,
                                  geojson=geojson_capital, 
                                  locations='BARRIO', 
                                  color='house',
                                  opacity=0.5, height=500,)['data'][0],
                                  row=2, col=1)

fig.add_trace(px.choropleth_mapbox(capital_merge_propiedades,
                                  geojson=geojson_capital, 
                                  locations='BARRIO', 
                                  color='store',
                                  opacity=0.5, height=500,)['data'][0],
                                  row=2, col=2)

fig.update_mapboxes(
        center = {"lat": -34.615, "lon": -58.45},
        zoom = 10,
        style='carto-positron',
 )

fig.update_layout(
    margin={"r":0,"t":20,"l":0,"b":0},
    height=950
)

fig.show()

In [ ]:
fig = px.choropleth_mapbox(capital_merge_propiedades, geojson=geojson_capital,
                      locations='BARRIO', 
                      color='PH',
                      color_continuous_scale="OrRd",
                      range_color=(0, capital_merge_propiedades['PH'].max()),
                      mapbox_style="carto-positron",
                      opacity=0.5,
                      zoom=10.5,
                    labels = {'PH':'Cantidad de PH'},
                      center = {"lat": -34.61, "lon": -58.45})

fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0})
fig.show()